# nanowire 1D SNS junctions with Majoranas

## Connect to cluster 

In [ ]:
import hpc05

client = hpc05.Client(profile="pbs3", timeout=60)
print("Connected to hpc05")

In [ ]:
dview = client[:]
dview.use_dill()
lview = client.load_balanced_view()
print("Connected to {} engines.".format(len(dview)))

In [ ]:
# %%px --local
# Standard library imports
from itertools import product
import functools
from types import SimpleNamespace
import pickle

# Related third party imports
import holoviews as hv
import kwant
from kwant.continuum.discretizer import discretize
import numpy as np
import pandas as pd
import scipy.sparse.linalg as sla
import scipy.optimize
import scipy.constants

d = SimpleNamespace(
    L=hv.Dimension(("L", "$L$"), unit="nm"),
    L_sc=hv.Dimension(("L_sc", "$L_{sc}$"), unit="nm"),
    T=hv.Dimension(("T", "$T$"), unit="K"),
    k=hv.Dimension(r"$k$", unit=r"nm$^{-1}$"),
    E=hv.Dimension(r"$E$", unit=r"meV"),
    phi=hv.Dimension(r"Phase $\phi$"),
    I=hv.Dimension(r"$I$", unit="nA"),
    B_x=hv.Dimension(("B_x", r"$B_x$"), unit="T"),
    mu=hv.Dimension(("mu", r"$\mu$"), unit="meV"),
    inv_xi=hv.Dimension(r"$1/\xi$", unit=r"$\mu m^{-1}$"),
    x=hv.Dimension("$x$", unit="nm"),
    I_c=hv.Dimension("$I_c$", unit="nA"),
)


def memoize(obj):
    # From https://wiki.python.org/moin/PythonDecoratorLibrary#Alternate_memoize_as_nested_functions
    cache = obj.cache = {}

    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = obj(*args, **kwargs)
        return cache[key]

    return memoizer


@memoize
def get_templates(a):
    ham = (
        "(0.5 * hbar**2 * k_x**2 / m_eff * c - mu) * kron(sigma_0, sigma_z) -"
        "alpha * k_x * kron(sigma_y, sigma_z) + "
        "0.5 * g * mu_B * B_x * kron(sigma_x, sigma_0) + Delta * kron(sigma_0, sigma_x)"
    )

    templ_normal = discretize(ham, locals={"Delta": 0}, grid_spacing=a)
    templ_sc = discretize(ham, grid_spacing=a)
    return templ_normal, templ_sc


@memoize
def make_1d_SNS(a=10, L=400, L_sc=400, with_leads=True):
    """Create a 1D semiconducting wire of length `L` with superconductors
    of length `L_sc` on its ends.

    Parameters
    ----------
    a : int
        Discretization constant in nm.
    L : int
        Length of wire (the scattering semi-conducting part) in nm.
    L_sc : int
        Length of superconducting ends in nm.
    with_leads : bool
        Add infinite SC leads to the ends of the nanowire.

    Returns
    -------
    syst : kwant.builder.FiniteSystem
        The finilized kwant system.
    hopping : function
        Function that returns the hopping matrix between the two cross sections
        of where the SelfEnergyLead is attached.
    """
    templ_normal, templ_sc = get_templates(a)

    lat = lat_from_temp(templ_normal)
    syst = kwant.Builder()

    def shape(x_left, x_right):
        return lambda s: x_left <= s.pos[0] < x_right, (x_left,)

    shape_left = shape(-L_sc, 0)
    shape_middle = shape(0, L)
    shape_right = shape(L, L + L_sc)
    syst.fill(templ_sc, *shape_left)
    syst.fill(
        templ_normal,
        lambda site: shape_left[0](site) or shape_middle[0](site),
        shape_middle[1],
    )
    syst.fill(
        templ_sc,
        lambda site: shape_middle[0](site) or shape_right[0](site),
        shape_right[1],
    )

    cuts = get_cuts(syst, lat, 0, 1)
    syst = add_vlead(syst, lat, *cuts)

    lead = kwant.Builder(kwant.TranslationalSymmetry([a]))
    lead.fill(templ_sc, lambda x: True, (0,))

    if with_leads:
        syst.attach_lead(lead)
        syst.attach_lead(lead.reversed())

    syst = syst.finalized()

    hopping = hopping_between_cuts(syst, *cuts)
    return syst, hopping


@memoize
def make_1d_wire(a=10, L=400, with_leads=False):
    template = get_templates(a)[1]
    lat = lat_from_temp(template)
    syst = kwant.Builder()

    def shape(x_left, x_right):
        return lambda s: x_left <= s.pos[0] <= x_right, (x_left,)

    syst.fill(template, *shape(0, L))

    cuts = get_cuts(syst, lat, L // (2 * a), (L // (2 * a) + 1))
    syst = add_vlead(syst, lat, *cuts)

    lead = kwant.Builder(kwant.TranslationalSymmetry([a]))
    lead.fill(template, lambda x: True, (0,))

    if with_leads:
        syst.attach_lead(lead)
        syst.attach_lead(lead.reversed())

    syst = syst.finalized()

    hopping = hopping_between_cuts(syst, *cuts)
    return syst, hopping


@memoize
def plot_bands(lead, params):
    """
    Plots the bandstructure for lead.

    Parameters:
    -----------
    p : SimpleNamespace object
        A simple container that is used to store Hamiltonian parameters.
    lead : kwant.builder.InfiniteSystem object
        The finalized infinite system.
    c : SimpleNamespace object
        A namespace container with all constant (fundamental) parameters used.
    """
    Q = (
        np.sqrt(params["mu"] ** 2 + params["Delta"] ** 2)
        - np.sqrt(params["B_x"] ** 2 + params["B_y"] ** 2 + params["B_z"] ** 2)
        * params["g"]
        * params["mu_B"]
        / 2
    )
    if Q > 0:
        print("System is trivial (B < sqrt(mu**2 + Delta**2))")
    else:
        print("System is topological (B > sqrt(mu**2 + Delta**2))")

    bands = kwant.physics.Bands(lead, params=params)
    momenta = np.linspace(-2, 2, 51)
    evs = [bands(k=k) for k in momenta]

    dims = [d.k, d.E]

    return hv.Path((momenta, evs), kdims=dims, label="Band structure")


@memoize
def plot_CPR(syst, hopping, params, tol=0.01, max_frequencies=1000, label=None):
    phases = np.linspace(-np.pi, np.pi, 51)
    H_0_cache = []
    I = [
        current_at_phase(syst, hopping, params, H_0_cache, phase, tol, max_frequencies)
        for phase in phases
    ]

    if label is None:
        label = "Nummerical CPR"

    return hv.Curve((phases, I), kdims=[d.phi], vdims=[d.I], label=label)


@memoize
def densities(vecs, num_orbital=4):
    if len(vecs.shape) == 1:
        return np.linalg.norm(vecs.reshape(-1, num_orbital), axis=1) ** 2
    else:
        return np.linalg.norm(vecs.reshape(-1, num_orbital, len(vecs)), axis=1) ** 2


@memoize
def find_lowest_energy_state(syst, params):
    """
    Finds the lowest energy state with positive energy of the finite system.

    Parameters:
    -----------
    syst : kwant.builder.FiniteSystem object
          The finalized system.

    Returns:
    --------
    wf_sq : numpy.ndarray
        The wavefunction squared.
    """
    ham = syst.hamiltonian_submatrix(sparse=True, params=params)
    ev, evec = sla.eigsh(ham, k=10, sigma=0)

    # find the one with the positive eigenvalue closest to 0
    ev_index1, ev_index2 = np.argsort(abs(np.array(ev)))[:2]
    if ev[ev_index1] > 0:
        ev_index = ev_index1
    else:
        ev_index = ev_index2

    wf = evec[:, ev_index]

    wf_sq = densities(wf)
    return wf_sq, ev[ev_index]


@memoize
def translation_ev(h, t, tol=1e6):
    """Compute the eigenvalues of the translation operator of a lead.

    Adapted from kwant.physics.leads.modes.

    Parameters
    ----------
    h : numpy array, real or complex, shape (N, N) The unit cell
        Hamiltonian of the lead unit cell.
    t : numpy array, real or complex, shape (N, M)
        The hopping matrix from a lead cell to the one on which self-energy
        has to be calculated (and any other hopping in the same direction).
    tol : float
        Numbers and differences are considered zero when they are smaller
        than `tol` times the machine precision.

    Returns
    -------
    ev : numpy array
        Eigenvalues of the translation operator in the form lambda=r*exp(i*k),
        for |r|=1 they are propagating modes.
    """
    a, b = kwant.physics.leads.setup_linsys(h, t, tol, None).eigenproblem
    ev = kwant.physics.leads.unified_eigenproblem(a, b, tol=tol)[0]
    return ev


def cell_mats(lead, params, bias=0):
    h = lead.cell_hamiltonian(params=params)
    h -= bias * np.identity(len(h))
    t = lead.inter_cell_hopping(params=params)
    return h, t


@memoize
def slowest_evan_mode(lead, params, a):
    """Find the slowest decaying (evanescent) mode.

    It uses an adapted version of the function kwant.physics.leads.modes,
    in such a way that it returns the eigenvalues of the translation operator
    (lamdba = |r|*e^ik). The imaginary part of the wavevector k, is the part that
    makes it decay. The inverse of this Im(k) is the size of a Majorana bound
    state. The norm of the eigenvalue that is closest to one is the slowes
    decaying mode. Also called decay length.

    Parameters:
    -----------
    lead : kwant.builder.InfiniteSystem object
        The finalized infinite system.

    Returns:
    --------
    majorana_length : float
        The length of the Majorana.
    """
    h, t = cell_mats(lead, params, bias=0)
    ev = translation_ev(h, t)
    norm = ev * ev.conj()
    idx = np.abs(norm - 1).argmin()
    majorana_length = np.abs(a / np.log(ev[idx]).real)
    return majorana_length


@memoize
def find_phase_bounds(lead, params, B, k=0, num_bands=5):
    """Find the phase boundaries.
    Solve an eigenproblem that finds values of chemical potential at which the
    gap closes at momentum k=0. We are looking for all real solutions of the
    form H\psi=0 so we solve sigma_0 * tau_z H * psi = mu * psi.
    Parameters:
    -----------
    lead : kwant.builder.InfiniteSystem object
        The finalized infinite system.
    p : types.SimpleNamespace object
        A simple container that is used to store Hamiltonian parameters.
    B : tuple of floats
        A tuple that contains magnetic field strength in x, y and z-directions.
    k : float
        Momentum value, by default set to 0.
    Returns:
    --------
    chemical_potential : numpy array
        Twenty values of chemical potential at which a bandgap closes at k=0.
    """

    params["B_x"], params["B_y"], params["B_z"] = B
    params["mu"] = 0
    h, t = cell_mats(lead, params)
    tk = t * np.exp(1j * k)
    h_k = h + tk + tk.T.conj()
    sigma_z = np.array([[1, 0], [0, -1]])
    chemical_potentials = np.kron(np.eye(len(h) // 2), sigma_z) @ h_k
    if num_bands is None:
        return np.linalg.eigvals(chemical_potentials)
    else:
        return sla.eigs(chemical_potentials, k=num_bands, sigma=0)[0]

In [ ]:
%matplotlib inline
hv.notebook_extension()
%output size=200
%opts Path [aspect='square']

# Helper `funcs.py` for the super currents

In [ ]:
# %%px --local
# Parameters taken from arXiv:1204.2792
# All constant parameters, mostly fundamental
# constants, in a types.SimpleNamespace.
constants = SimpleNamespace(
    m_eff=0.015 * scipy.constants.m_e,  # effective mass in kg
    hbar=scipy.constants.hbar,
    m_e=scipy.constants.m_e,
    eV=scipy.constants.eV,
    e=scipy.constants.e,
    meV=scipy.constants.eV * 1e-3,
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    current_unit=scipy.constants.k
    * scipy.constants.e
    / scipy.constants.hbar
    * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants["Bohr magneton"][0]
    / (scipy.constants.eV * 1e-3),
    t=scipy.constants.hbar**2
    / (2 * 0.015 * scipy.constants.m_e)
    / (scipy.constants.eV * 1e-3 * 1e-18),
    c=1e18 / (scipy.constants.eV * 1e-3),
)


def lat_from_temp(template):
    return next(iter(template.sites())).family


def lattice_constant_from_syst(syst):
    lat = lat_from_syst(syst)
    a = np.max(lat.prim_vecs)
    return a


@memoize
def I_c(syst, hopping, params, tol=1e-2, max_frequencies=500, N_brute=30):
    """Find the critical current by optimizing the current-phase
    relation.

    Parameters
    ----------
    syst : kwant.builder.FiniteSystem
        The finilized kwant system.
    hopping : function
        Function that returns the hopping matrix between the two cross
        sections of where the SelfEnergyLead is attached.
    params : dict
        A container that is used to store Hamiltonian parameters.
    tol : float, optional
        Tolerance of the `current_at_phase` function.
    max_frequencies : int, optional
        Maximum number of Matsubara frequencies.
    N_brute : int, optional
        Number of points at which the CPR is evaluated in the brute
        force part of the algorithm.

    Returns
    -------
    dict
        Dictionary with the critical phase, critical current, and `currents`
        evaluated at `phases`."""
    H_0_cache = []
    func = lambda phase: -current_at_phase(
        syst, hopping, params, H_0_cache, phase, tol, max_frequencies
    )
    opt = scipy.optimize.brute(
        func, ranges=((-np.pi, np.pi),), Ns=N_brute, full_output=True
    )
    x0, fval, grid, Jout = opt
    return dict(
        phase_c=x0[0],
        current_c=-fval,
        phases=grid,
        currents=-Jout,
        N_freqs=len(H_0_cache),
    )


def get_cuts(syst, lat, x_left=0, x_right=1):
    """Get the sites at two postions of the specified cut coordinates.

    Parameters
    ----------
    syst : kwant.builder.FiniteSystem
        The finilized kwant system.
    lat : dict
        A container that is used to store Hamiltonian parameters.
    """
    l_cut = [lat(*tag) for tag in [s.tag for s in syst.sites()] if tag[0] == x_left]
    r_cut = [lat(*tag) for tag in [s.tag for s in syst.sites()] if tag[0] == x_right]
    assert len(l_cut) == len(r_cut), "x_left and x_right use site.tag not site.pos!"
    return l_cut, r_cut


def add_vlead(syst, lat, l_cut, r_cut):
    dim = lat.norbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(
        lambda energy, args: np.zeros((dim, dim)), l_cut + r_cut
    )
    syst.leads.append(vlead)
    return syst


def hopping_between_cuts(syst, r_cut, l_cut):
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]

    def hopping(syst, params):
        return syst.hamiltonian_submatrix(
            params=params, to_sites=l_cut_sites, from_sites=r_cut_sites
        )[::2, ::2]

    return hopping


@memoize
def current_at_phase(
    syst, hopping, params, H_0_cache, phase, tol=1e-2, max_frequencies=500
):
    """Find the supercurrent at a phase using a list of Hamiltonians at
    different imaginary energies (Matsubara frequencies). If this list
    does not contain enough Hamiltonians to converge, it automatically
    appends them at higher Matsubara frequencies untill the contribution
    is lower than `tol`, however, it cannot exceed `max_frequencies`.

    Parameters
    ----------
    syst : kwant.builder.FiniteSystem
        The finilized kwant system.
    hopping : function
        Function that returns the hopping matrix between the two cross sections
        of where the SelfEnergyLead is attached.
    params : dict
        A container that is used to store Hamiltonian parameters.
    H_0_cache : list
        Hamiltonians at different imaginary energies.
    phase : float, optional
        Phase at which the supercurrent is calculated.
    tol : float, optional
        Tolerance of the `current_at_phase` function.
    max_frequencies : int, optional
        Maximum number of Matsubara frequencies.

    Returns
    -------
    dict
        Dictionary with the critical phase, critical current, and `currents`
        evaluated at `phases`."""

    H12 = hopping(syst, params)
    I = 0
    for n in range(max_frequencies):
        if len(H_0_cache) <= n:
            H_0_cache.append(null_H(syst, params, n))
        I_contrib = current_contrib_from_H_0(H_0_cache[n], H12, phase, params)
        I += I_contrib
        if I_contrib == 0 or tol is not None and abs(I_contrib / I) < tol:
            return I
    # Did not converge within tol using max_frequencies Matsubara frequencies.
    if tol is not None:
        return np.nan
    # if tol is None, return the value after max_frequencies is reached.
    else:
        return I


def lat_from_syst(syst):
    lats = set(s.family for s in syst.sites)
    if len(lats) > 1:
        raise Exception("No unique lattice in the system.")
    return list(lats)[0]


def matsubara_frequency(n, params):
    """n-th fermionic Matsubara frequency at temperature T.

    Parameters
    ----------
    n : int
        n-th Matsubara frequency

    Returns
    -------
    float
        Imaginary energy.
    """
    return (2 * n + 1) * np.pi * params["k"] * params["T"] * 1j


def null_H(syst, params, n):
    """Return the Hamiltonian (inverse of the Green's function) of
    the electron part at zero phase.

    Parameters
    ----------
    syst : kwant.builder.FiniteSystem
        The finilized kwant system.
    params : dict
        A container that is used to store Hamiltonian parameters.
    n : int
        n-th Matsubara frequency

    Returns
    -------
    numpy.array
        The Hamiltonian at zero energy and zero phase."""
    en = matsubara_frequency(n, params)
    gf = kwant.greens_function(
        syst, en, out_leads=[0], in_leads=[0], check_hermiticity=False, params=params
    )
    return np.linalg.inv(gf.data[::2, ::2])


def gf_from_H_0(H_0, t):
    """Returns the Green's function at a phase that is defined inside `t`.
    See doc-string of `current_from_H_0`.
    """
    H = np.copy(H_0)
    dim = t.shape[0]
    H[:dim, dim:] -= t.T.conj()
    H[dim:, :dim] -= t
    return np.linalg.inv(H)


def current_from_H_0(H_0_cache, H12, phase, params):
    """Uses Dyson’s equation to obtain the Hamiltonian for other
    values of `phase` without further inversions (calling `null_H`).

    Parameters
    ----------
    H_0_cache : list
        Hamiltonians at different imaginary energies.
    H12 : numpy array
        The hopping matrix between the two cross
        sections of where the SelfEnergyLead is attached.
    phase : float
        Phase at which the supercurrent is calculated.
    params : dict
        A container that is used to store Hamiltonian parameters.

    Returns
    -------
    float
        Total current of all terms in `H_0_list`.
    """
    I = sum(current_contrib_from_H_0(H_0, H12, phase, params) for H_0 in H_0_cache)
    return I


def current_contrib_from_H_0(H_0, H12, phase, params):
    """Uses Dyson’s equation to obtain the Hamiltonian for other
    values of `phase` without further inversions (calling `null_H`).

    Parameters
    ----------
    H_0 : list
        Hamiltonian at a certain imaginary energy.
    H12 : numpy array
        The hopping matrix between the two cross
        sections of where the SelfEnergyLead is attached.
    phase : float
        Phase at which the supercurrent is calculated.
    params : dict
        A container that is used to store Hamiltonian parameters.
    Returns
    -------
    float
        Current contribution of `H_0`.
    """
    t = H12 * np.exp(1j * phase)
    gf = gf_from_H_0(H_0, t - H12)
    dim = t.shape[0]
    H12G21 = t.T.conj() @ gf[dim:, :dim]
    H21G12 = t @ gf[:dim, dim:]
    return (
        -4
        * params["T"]
        * params["current_unit"]
        * (np.trace(H21G12) - np.trace(H12G21)).imag
    )

# Plot an example system and show $\Delta$

In [ ]:
syst, hopping = make_1d_SNS(a=10, L=100, L_sc=200, with_leads=False)
lead = make_1d_wire(a=10, L=100, with_leads=True)[0].leads[1]

params = dict(
    T=0.001,
    Delta=0.250,
    mu=0.01,
    B_x=0.23,
    B_y=0,
    B_z=0,
    g=50,
    alpha=20,
    **constants.__dict__
)


def delta(sites):
    return [
        np.abs(syst.hamiltonian(i, i, params=params)[1, 0])
        for i, site in enumerate(sites)
    ]


kwant.plot(syst, site_color=delta(syst.sites), fig_size=(10, 10));

# Example CPR


In [ ]:
%%time
plot_CPR(syst, hopping, params)

# Phase diagram

In [ ]:
Bs = np.linspace(0, 1)
bounds = np.sort(
    [find_phase_bounds(lead, params, (B_x, 0, 0), num_bands=None).real for B_x in Bs],
    axis=1,
)
hv.Path(
    (Bs, np.array(bounds)), kdims=[d.B_x, d.mu], label="Topological phase boundaries"
)

# Majorana decay length $\xi$

In [ ]:
%%opts Image [colorbar=True aspect='square']

mus = np.linspace(-0.5, 0.5, 51)
Bs = np.linspace(0, 2, 51)
xis = np.array([[1000 / slowest_evan_mode(lead, params, 10) for params['B_x'] in Bs] for params['mu'] in mus])
im = hv.Image(np.flipud(xis), kdims=[d.B_x, d.mu],
              vdims=[d.inv_xi],
              bounds=(Bs.min(), mus.min(), Bs.max(), mus.max()),
              label='Inverse decay length')

hm = hv.HoloMap({mu: im * hv.HLine(mu) + im.sample(mu=mu) for mu in mus[1:-1]}, kdims=[r'$\mu$']).collate()
hm

In [ ]:
%%opts Image [colorbar=True aspect='square']
params['alpha'] *= 5
mus = np.linspace(-0.5, 0.5, 51)
Bs = np.linspace(0, 2, 51)
xis = np.array([[1000 / slowest_evan_mode(lead, params, 10) for params['B_x'] in Bs] for params['mu'] in mus])
im = hv.Image(np.flipud(xis), kdims=[d.B_x, d.mu],
              vdims=[d.inv_xi],
              bounds=(Bs.min(), mus.min(), Bs.max(), mus.max()),
              label='Inverse decay length')

hm = hv.HoloMap({mu: im * hv.HLine(mu) + im.sample(mu=mu) for mu in mus[1:-1]}, kdims=[r'$\mu$']).collate()
hm

In [ ]:
print("Longest Majorana at:")
i, j = np.unravel_index(xis.argmin(), xis.shape)
print("B_x={}, mu={}, xi = {} µm".format(Bs[i], mus[j], 1 / xis[i, j]))
print()
# print('Longest Majoranas at these values:')
# for mu, row in zip(mus, xis.T):
#     print('mu={}, B_x={}'.format(mu, Bs[np.argmin(row)]))

# Plotting the Majorana wavefunction

In [ ]:
def wf(B_x, mu, params=params):
    params["B_x"] = B_x
    params["mu"] = mu

    wf_sq, energy = find_lowest_energy_state(syst, params)
    xi = slowest_evan_mode(lead, params, 10)
    a = np.max(lat_from_syst(lead).prim_vecs)

    sites = np.arange(syst.sites[0].pos[0], syst.sites[-1].pos[0] + a, a)
    label = "E={:.2f} µeV, xi={:.2f} µm".format(1000 * energy, xi / 1000)
    return hv.Path((sites, wf_sq), kdims=[d.x, d.E], label=label)


dmap = hv.DynamicMap(
    wf,
    kdims=[
        hv.Dimension("B_x", values=np.arange(0, 0.5, 0.01)),
        hv.Dimension("mu", values=np.arange(0, 1, 0.01)),
    ],
)
dmap

In [ ]:
wf(B_x=0.25, mu=0.01)

# Majorana spectrum

In [ ]:
@memoize
def get_energies(syst, params):
    H = syst.hamiltonian_submatrix(params=params)
    return np.linalg.eigvalsh(H)

In [ ]:
%%opts Path [aspect='square']

syst, hopping = make_1d_wire(a=10, L=3000, with_leads=False)
lead = make_1d_wire(a=10, L=10, with_leads=True)[0].leads[1]

params = dict(Delta=0.250, B_x=0.25, B_y=0, B_z=0, g=50, alpha=20, **constants.__dict__)

mus = np.linspace(0, 0.5)
energies = [get_energies(syst, params) for params['mu'] in mus]

hv.Path((mus, energies), kdims=[d.mu, d.E])[:, -.5:0.5]

# Interactive bandstructure

In [ ]:
%%opts Path [aspect='square']

kdims = [hv.Dimension("ylim", range=(0.05, 50)), 
         hv.Dimension(r"$\alpha$", range=(0, 50), unit='meV·nm'), 
         hv.Dimension(r"$\mu$", range=(0.0, 30), unit='meV'), 
         hv.Dimension(r"$B_x$", range=(0, 10), unit='T'), 
         hv.Dimension(r"$\Delta$", range=(0, 1.), unit='meV')]

def dm(lead, params, ylim, alpha, mu, B_x, Delta):
    """Simple wrapper function."""
    params['mu'] = mu
    params['alpha'] = alpha
    params['B_x'] = B_x
    params['Delta'] = Delta
    return plot_bands(lead, params).select(E=(-ylim, ylim))

hv.DynamicMap(functools.partial(dm, lead, params), kdims=kdims)

# SNS junctions with multiple Majoranas

## Current at $B_x=0$

In [ ]:
params = dict(
    Delta=0.250, B_x=0.25, mu=0.01, g=50, alpha=20, T=0.1, **constants.__dict__
)
Ls = [10, 100, 300, 500, 1000, 1500, 2000, 2500, 3000]
Ts = [0.001, 0.01, 0.1, 1]
hm = {}
for L in Ls:
    for T in Ts:
        params["T"] = T
        syst, hopping = make_1d_SNS(a=10, L=L, L_sc=3000, with_leads=False)
        hm[(L, T)] = plot_CPR(syst, hopping, params)

In [ ]:
hv.HoloMap(hm, kdims=[d.L, d.T]).overlay("L").layout("T").cols(2)

In [ ]:
hv.HoloMap(hm, kdims=[d.L, d.T]).overlay("T")

## Critical current as function of $B_x$: $I_c(B_x)$

In [ ]:
# params = dict(Delta=0.250, B_x=0.25, mu=0.01, g=50, alpha=20, T=0.1, **constants.__dict__)
# Ls = [10, 100, 300, 500, 1000]
# Ts = [0.01, 0.1, 1]
# B_xs = np.linspace(0, 1, 51)
# hm = {}
# currents = {}
# for with_leads in [True]:
#     for L in Ls:
#         for T in Ts:
#             params['T'] = T
#             L_sc = 10 if with_leads else 1000
#             syst, hopping = make_1d_SNS(a=10, L=L, L_sc=L_sc, with_leads=with_leads)
#             current = [I_c(syst, hopping, params)['current_c'] for params['B_x'] in B_xs]
#             label = 'T={}'.format(T)
#             currents[(with_leads, L, T)] = hv.Path((B_xs, current), kdims=[d.B_x, d.I_c], label=label)

# Do it parallel

In [ ]:
from itertools import repeat

Ls = [20, 50, 100, 300, 500, 1000]
Ts = [0.001, 0.01, 0.1, 1]
with_leadss = [True, False]
L_scs = [20, 50, 100, 200, 300, 500, 1000]
B_xs = np.linspace(0, 1, 101)


def get_currents(x, constants=constants, B_xs=B_xs):
    L, L_sc, with_leads, T = x
    params = dict(Delta=0.250, mu=0.01, g=50, alpha=20, T=T, **constants.__dict__)
    if with_leads:
        L_sc = 10
    syst, hopping = make_1d_SNS(a=10, L=L, L_sc=L_sc, with_leads=with_leads)
    currents = [I_c(syst, hopping, params)["current_c"] for params["B_x"] in B_xs]
    return currents


vals = list(product(Ls, L_scs, with_leadss, Ts))
print(len(vals))

# results = lview.map_async(get_currents, vals)
# # results.wait_interactive()
# currents = results.result()

# fname = 'currents_1d_SNS.p'

# with open(fname, 'wb') as pfile:
#     pickle.dump(currents, pfile)

# with open(fname, 'rb') as f:
#     test = pickle.load(f)

In [ ]:
fname = "currents_1d_SNS.p"
with open(fname, "rb") as f:
    test = pickle.load(f)

In [ ]:
hm = {}
for i, (L, L_sc, with_leads, T) in enumerate(vals):
    label = "T={}".format(T)
    hm[(with_leads, L, L_sc, T)] = hv.Path(
        (B_xs, test[i]), kdims=[d.B_x, d.I_c], label=label
    )

In [ ]:
%%opts Path {+framewise} [show_legend=True] Overlay [show_legend=True] 
kdims = ['with_leads', d.L, d.L_sc, d.T]
B_c = np.sqrt(params['mu']**2 + params['Delta']**2) / (params['g'] * params['mu_B'] / 2)
hv.HoloMap(hm, kdims=kdims).overlay('T').select(B_x=(0, 1)) * hv.VLine(B_c)